<a href="https://colab.research.google.com/github/varnikakothari/Forecasting-the-SP-500-Comparative-AI-Analysis-of-Macroeconomic-and-Commodity-Indicators/blob/main/Forecasting_the_S%26P_500_Comparative_AI_Analysis_of_Macroeconomic_and_Commodity_Indicators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Forecasting the S&P 500: A Comparative Analysis of Traditional and Non-Linear Macroeconomic Indicators and Commodity Linkages Using Artificial Intelligence Models

Varnika Kothari, Rami Abi Akl

Datasets Used:

US Stock Market Data: S&P 500 Index (1901-2025):
Author: Ahmadul Karim Chowdhury
https://www.kaggle.com/datasets/ahmadulkc/s-and-p-500-historical-monthly-prices-19012025

Crude Oil Price (1983-Present):
Author: Chandan Singh
License: CCO: Public Domain
https://www.kaggle.com/datasets/sc231997/crude-oil-price

Gold, Silver & Precious Metals Futures (2000-2024):
Author: Guillem SD
License: Attribution-NonCommercial 4.0 International
https://www.kaggle.com/datasets/guillemservera/precious-metals-data

USA Macroeconomic Rate of Changes (1993-2025):
Data collected from FRED-Federal Reserve Economic Data from the Bank of St.Louis
https://fred.stlouisfed.org/
Compiled Dataset:- https://docs.google.com/spreadsheets/d/10q3gGOYpTtAfJsDukJU2x3SYuyGlvYIq/edit?usp=sharing&ouid=117010112379515535428&rtpof=true&sd=true


*Note that all model results print MSE that is Mean Squared Error which is then mathematically square rooted to obtain RMSE or Root Mean Squared Error

#USA S&P 500 Dataset

In [ ]:
#To import the dataset and create a new one with only data after 1993

import pandas as pd
import kagglehub
import datetime

FILE_PATH = "S_P 500 Historical Data.csv"
NEW_FILE_NAME = "SP500_1993_onwards.csv"
START_DATE_FILTER = '1993-01-01'

df_SP = kagglehub.load_dataset(
    kagglehub.KaggleDatasetAdapter.PANDAS,
    "ahmadulkc/s-and-p-500-historical-monthly-prices-19012025",
    FILE_PATH,
    pandas_kwargs={
        'thousands': ',',
    }
)
df_SP['Date'] = pd.to_datetime(df_SP['Date'], format='%m-%d-%y', errors='coerce')

df_SP['Date'] = df_SP['Date'].apply(lambda x: x.replace(year=x.year - 100)
                                    if pd.notna(x) and x.year < 1993
                                    else x)

df_SP['Change %'] = df_SP['Change %'].astype(str).str.replace('%', '', regex=False)
df_SP['Change %'] = pd.to_numeric(df_SP['Change %'], errors='coerce') / 100

numeric_cols_to_check = ['Price', 'Open', 'High', 'Low']
for col in numeric_cols_to_check:
    if col in df_SP.columns and df_SP[col].dtype == 'object':
        df_SP[col] = pd.to_numeric(df_SP[col], errors='coerce')

df_1993_onwards = df_SP[df_SP['Date'] >= START_DATE_FILTER].copy()

df_1993_onwards = df_1993_onwards.drop(columns=['Open', 'High','Low'])

In [ ]:
df_1993_onwards['Date'] = pd.to_datetime(df_1993_onwards['Date'])

df_1993_onwards['Year'] = df_1993_onwards['Date'].dt.year
df_1993_onwards['Month'] = df_1993_onwards['Date'].dt.month

In [ ]:
df_1993_onwards.head()

#Crude Oil


In [ ]:
#To import the dataset
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "crude-oil-price.csv"

df_Crude = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "sc231997/crude-oil-price",
  file_path,
)


#Gold, Silver

In [ ]:
#To import the dataset
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "all_commodities_data.csv"
df_commodity = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "guillemservera/precious-metals-data",
  file_path,
)

In [ ]:
df_commodity.head()

#Macroeconomic Indicators


In [ ]:
#Please download Macro.xlsx using the link given at the beginning and save it in My Drive
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount("/content/drive")

df_indicators = pd.read_excel("/content/drive/My Drive/Macro.xlsx")


In [ ]:
df_indicators['Year'] = df_indicators['observation_date'].dt.year
df_indicators['Month'] = df_indicators['observation_date'].dt.month

df_indicators.head()

In [ ]:
df_indicators.head()

#Final Data Set Creation

In [ ]:
#The dataset created in this code bloc is only used for testing the RNN Models and is simply a theoretical representation as it assumes SP500 Prices remain constant throughout a month

import pandas as pd

# --- Preprocessing for df_1993_onwards (SP500 data) ---
df_1993_onwards['Date'] = pd.to_datetime(df_1993_onwards['Date'])
df_1993_onwards['Year'] = df_1993_onwards['Date'].dt.year
df_1993_onwards['Month'] = df_1993_onwards['Date'].dt.month
# Rename the 'Price' column for SP500
df_1993_onwards = df_1993_onwards.rename(columns={'Price': 'SP500_Price'})


# --- Preprocessing for df_Crude ---
df_Crude['date'] = pd.to_datetime(df_Crude['date'])
df_Crude['Year'] = df_Crude['date'].dt.year
df_Crude['Month'] = df_Crude['date'].dt.month
# Rename the 'price' column for Crude (optional, but good for consistency)
df_Crude = df_Crude.rename(columns={'price': 'Crude_Price'})


# --- Preprocessing for df_commodity (Gold data) ---
# 1. Filter for 'Gold' commodity
df_gold_commodity = df_commodity[df_commodity['commodity'] == 'Gold'].copy()
df_silver_commodity = df_commodity[df_commodity['commodity'] == 'Silver'].copy()
df_gold_commodity = df_gold_commodity.rename(columns={'close': 'Gold_Price'})
df_silver_commodity = df_silver_commodity.rename(columns={'close': 'Silver_Price'})
# 2. Convert 'date' to datetime and extract 'Year' and 'Month'
df_gold_commodity['date'] = pd.to_datetime(df_gold_commodity['date'])
df_gold_commodity['Year'] = df_gold_commodity['date'].dt.year
df_gold_commodity['Month'] = df_gold_commodity['date'].dt.month
df_silver_commodity['date'] = pd.to_datetime(df_silver_commodity['date'])
df_silver_commodity['Year'] = df_silver_commodity['date'].dt.year
df_silver_commodity['Month'] = df_silver_commodity['date'].dt.month

# --- Perform sequential merges ---

# First merge: df_1993_onwards (SP500) with df_indicators
merged_df_step1 = pd.merge(df_1993_onwards, df_indicators, on=['Year', 'Month'], how='inner')

# Second merge: result of step 1 with df_Crude
merged_df_step2 = pd.merge(merged_df_step1, df_Crude, on=['Year', 'Month'], how='inner')

# Final merge: result of step 2 with the filtered df_gold_commodity
merged_df_step3 = pd.merge(merged_df_step2, df_gold_commodity, on=['Year', 'Month'], how='inner')
merged_df = pd.merge(merged_df_step3, df_silver_commodity, on=['Year', 'Month'], how='inner')
merged_df.head()

merged_df['SP500_NextPrice']=merged_df['SP500_Price'].shift(-1)
merged_df=merged_df.iloc[:-1]


In [ ]:
merged_df.head()

In [ ]:

#This is the dataset used for testing other models except RNN, and consists of SP500 Prices for the 1st date of every month.

# --- Preprocessing for df_commodity (Gold data) ---

# 2. Convert 'date' to datetime and extract 'Year' and 'Month'
monthly_avg_gold = df_gold_commodity.groupby(df_gold_commodity["date"].dt.to_period("M"))["Gold_Price"].mean().reset_index()
monthly_avg_gold['Year'] = monthly_avg_gold['date'].dt.year
monthly_avg_gold['Month'] = monthly_avg_gold['date'].dt.month
monthly_avg_silver = df_silver_commodity.groupby(df_silver_commodity["date"].dt.to_period("M"))["Silver_Price"].mean().reset_index()
monthly_avg_silver['Year'] = monthly_avg_silver['date'].dt.year
monthly_avg_silver['Month'] = monthly_avg_silver['date'].dt.month

# --- Perform sequential merges ---
# Final merge: result of step 2 with the filtered df_gold_commodity
merged_df_small = pd.merge(merged_df_step2, monthly_avg_gold, on=['Year', 'Month'], how='inner')
smallmerged_df = pd.merge(merged_df_small, monthly_avg_silver, on=['Year', 'Month'], how='inner')
smallmerged_df['SP500_NextPrice']=smallmerged_df['SP500_Price'].shift(-1)
smallmerged_df=smallmerged_df.iloc[:-1]
smallmerged_df.head()




#Correlation Matrices

In [ ]:
# --- 4. Select numeric columns for correlation ---
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
df_numeric = smallmerged_df.select_dtypes(include=['number'])

# --- 5. Calculate Correlation Matrix ---
corr_matrix = df_numeric.corr()

# --- 6. Define columns for the heatmap (explicitly include all desired columns) ---
# Ensure these columns exist in your df_numeric after merging your actual data
columns_for_heatmap = [
    'SP500_Price',
    'Crude_Price',
    'Gold_Price',
    'Silver_Price',
    'Industrial_Production',
    'Consumer_Price_Index',
    'Unemployment_Rate',
    'Retail_Sales',
    'Producer_Price_Index',
    'Personal_Consumption_Expenditures',
    'National_Home_Price_Index',
    'All_Employees(Total_Nonfarm)',
    'Labor_Force_Participation_Rate',
    'Money_Supply_(M2)',
    'Personal_Income',
    'Trade_Balance',
    'Consumer_Sentiment',]

# Filter the list to only include columns that actually exist in the correlation matrix
existing_columns_for_heatmap = [col for col in columns_for_heatmap if col in corr_matrix.columns]

# Ensure there are enough columns to plot a heatmap
subset_corr = corr_matrix.loc[existing_columns_for_heatmap, existing_columns_for_heatmap]

    # --- 7. Plot the Heatmap ---
plt.figure(figsize=[60, 60]) # Increased size for more columns
sns.heatmap(subset_corr, cmap='coolwarm', annot=True, fmt=".2f", linewidths=.9, annot_kws={"size": 40}) # Smaller annotation font
plt.title("Correlation Matrix of Financial and Economic Indicators", fontsize=45)
plt.xticks(rotation=90, ha='right', fontsize=40) # Rotate and adjust font for readability
plt.yticks(rotation=0, fontsize=40)
plt.tight_layout()



# Plots


In [ ]:

plt.figure(figsize=(2, 2))
plt.scatter(merged_df['Crude_Price'], merged_df['SP500_Price'], marker='o')
plt.ylabel('SP500 Price')
plt.xlabel('Crude_Price')
plt.grid(True)
plt.show()

In [ ]:

plt.figure(figsize=(2, 2))
plt.scatter(merged_df['Gold_Price'], merged_df['SP500_Price'], marker='o')
plt.ylabel('SP500 Price')
plt.xlabel('Gold_Price')
plt.grid(True)
plt.show()

In [ ]:

plt.figure(figsize=(2, 2))
plt.scatter(merged_df['Silver_Price'], merged_df['SP500_Price'], marker='o')
plt.ylabel('SP500 Price')
plt.xlabel('Silver_Price')
plt.grid(True)
plt.show()

In [ ]:

plt.figure(figsize=(2, 2))
plt.scatter(merged_df['Industrial_Production'], merged_df['SP500_Price'], marker='o')
plt.ylabel('SP500 Price')
plt.xlabel('Industrial_Production')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(2, 2))
plt.scatter(merged_df['Consumer_Price_Index'], merged_df['SP500_Price'], marker='o')
plt.ylabel('SP500 Price')
plt.xlabel('Consumer Price Index')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(2, 2))
plt.scatter(merged_df['Unemployment_Rate'], merged_df['SP500_Price'], marker='o')
plt.ylabel('SP500 Price')
plt.xlabel('Unemployment_Rate')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(2, 2))
plt.scatter(merged_df['Retail_Sales'], merged_df['SP500_Price'], marker='o')
plt.ylabel('SP500 Price')
plt.xlabel('Retail_Sales')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(2, 2))
plt.scatter(merged_df['Producer_Price_Index'], merged_df['SP500_Price'], marker='o')
plt.ylabel('SP500 Price')
plt.xlabel('Producer_Price_Index')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(2, 2))
plt.scatter(merged_df['Personal_Consumption_Expenditures'], merged_df['SP500_Price'], marker='o')
plt.ylabel('SP500 Price')
plt.xlabel('Personal Consumption')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(2, 2))
plt.scatter(merged_df['National_Home_Price_Index'], merged_df['SP500_Price'], marker='o')
plt.ylabel('SP500 Price')
plt.xlabel('National Home Price')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(2, 2))
plt.scatter(merged_df['All_Employees(Total_Nonfarm)'], merged_df['SP500_Price'], marker='o')
plt.ylabel('SP500 Price')
plt.xlabel('All_Employees')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(2, 2))
plt.scatter(merged_df['Labor_Force_Participation_Rate'], merged_df['SP500_Price'], marker='o')
plt.ylabel('SP500 Price')
plt.xlabel('Labour Participation')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(2, 2))
plt.scatter(merged_df['Money_Supply_(M2)'], merged_df['SP500_Price'], marker='o')
plt.ylabel('SP500 Price')
plt.xlabel('Money Supply')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(2, 2))
plt.scatter(merged_df['Trade_Balance'], merged_df['SP500_Price'], marker='o')
plt.ylabel('SP500 Price')
plt.xlabel('Trade_Balance')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(2, 2))
plt.scatter(merged_df['Personal_Income'], merged_df['SP500_Price'], marker='o')
plt.ylabel('SP500 Price')
plt.xlabel('Personal_Income')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(2, 2))
plt.scatter(merged_df['Consumer_Sentiment'], merged_df['SP500_Price'], marker='o')
plt.ylabel('SP500 Price')
plt.xlabel('Consumer_Sentiment')
plt.grid(True)
plt.show()

# Linear Regression

Feature sets can be modified by adding or subtracting desired features in the code where X is declared

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
X = smallmerged_df[[
'Crude_Price',
'Gold_Price',
'Silver_Price'

]]
y = smallmerged_df['SP500_NextPrice']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1
)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))

In [ ]:
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted')
plt.show()

# Decision Tree

Feature sets can be modified by adding or subtracting desired features in the code where X is declared

In [ ]:
from sklearn.tree import DecisionTreeRegressor,plot_tree

X = smallmerged_df[[
'Crude_Price',
]]
y = smallmerged_df['SP500_NextPrice']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1
)

decisionmodel = DecisionTreeRegressor(max_depth=50)
decisionmodel.fit(X_train, y_train)
y_predd = decisionmodel.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_predd))
print("R² Score:", r2_score(y_test, y_predd))



In [ ]:
plt.figure(figsize=(15,8))
plot_tree(decisionmodel, filled=True, feature_names=X_train.columns)
plt.show()

In [ ]:
plt.scatter(y_test, y_predd)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted')
plt.show()

#Random Forest

Feature sets can be modified by adding or subtracting desired features in the code where X is declared

In [ ]:
from sklearn.ensemble import RandomForestRegressor

X = smallmerged_df[[
'National_Home_Price_Index',
'Personal_Income',
'Personal_Consumption_Expenditures'
]]
y = smallmerged_df['SP500_NextPrice']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1
)

forestmodel = RandomForestRegressor(max_depth=30, random_state=1,n_estimators=5
)
forestmodel.fit(X_train, y_train)
y_predf = forestmodel.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_predf))
print("R² Score:", r2_score(y_test, y_predf))

In [ ]:
plt.scatter(y_test, y_predf)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted')
plt.show()

#Multilayer Perceptron Regressor


Feature sets can be modified by adding or subtracting desired features in the code where X is declared

In [ ]:
X = smallmerged_df[[

 'Unemployment_Rate',
 'Retail_Sales',
 'Producer_Price_Index',
 'Crude_Price',

]]
y = smallmerged_df['SP500_NextPrice']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1
)

In [ ]:
from sklearn.neural_network import MLPRegressor

MLP=MLPRegressor(hidden_layer_sizes=(128,64,32), activation='relu', solver='adam', max_iter=4000, random_state=1)
MLP.fit(X_train, y_train)
y_predm = MLP.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_predm))
print("R² Score:", r2_score(y_test, y_predm))

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.scatter(y_test, y_predm)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted')
plt.show()

The following code is used to test multiple combinations of structures for the same feature sets



In [ ]:
from itertools import combinations
values = [256, 128, 64, 32, 16, 8, 4,2]
lists = list(combinations(values, 4))
for lst in lists:
  MLP=MLPRegressor(hidden_layer_sizes=lst, activation='relu', solver='adam', max_iter=4000, random_state=1)
  MLP.fit(X_train, y_train)
  y_predm = MLP.predict(X_test)
  print(lst)
  print("Mean Squared Error:", mean_squared_error(y_test, y_predm))
  print("R² Score:", r2_score(y_test, y_predm))

#Recurrent Neural Network





Feature sets can be modified by adding or subtracting desired features in the code where X is declared

In [ ]:
#Setup
import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, callbacks, Sequential

# ---- CONFIGURATION that can be tweaked ----
LOOKBACK = 14
HORIZON = 1
VAL_RATIO = 0.15
TEST_RATIO = 0.15
UNITS = 32
BATCH = 16
EPOCHS = 200
PATIENCE = 20


merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df = merged_df.sort_values('date').reset_index(drop=True)


X = merged_df[[
'National_Home_Price_Index',
'Personal_Income',
'Personal_Consumption_Expenditures'
]]
y = merged_df[['SP500_NextPrice']].astype(float).values

X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.2, random_state=1
)

X_raw=X
y_raw=y

# 2) Scale features (and optionally y for stability)
x_scaler = StandardScaler().fit(X_raw)
y_scaler = StandardScaler().fit(y_raw)
X_scaled = x_scaler.transform(X_raw)
y_scaled = y_scaler.transform(y_raw)

# 3) Build sliding windows: shape (num_samples, LOOKBACK, num_features)
def make_windows(X, y, lookback=LOOKBACK, horizon=HORIZON):
    Xw, yw = [], []
    for i in range(len(X) - lookback - horizon + 1):
        Xw.append(X[i:i+lookback])
        yw.append(y[i+lookback+horizon-1])  # last point of horizon
    return np.array(Xw), np.array(yw)

Xw, yw = make_windows(X_scaled, y_scaled)

# 4) Chronological split (no shuffling)
n = len(Xw)
n_test  = int(np.floor(TEST_RATIO * n))
n_val   = int(np.floor(VAL_RATIO  * n))
n_train = n - n_val - n_test

X_train, y_train = Xw[:n_train], yw[:n_train]
X_val,   y_val   = Xw[n_train:n_train+n_val], yw[n_train:n_train+n_val]
X_test,  y_test  = Xw[n_train+n_val:],        yw[n_train+n_val:]

# 5) tf.data pipelines (optional but nice)
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH).prefetch(1)
val_ds   = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH).prefetch(1)
test_ds  = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH)

# 6) Define a tiny GRU model (swap to LSTM if you prefer)
model = Sequential([
    layers.Input(shape=(LOOKBACK, X_train.shape[-1])),
    layers.GRU(UNITS, dropout=0.2, recurrent_dropout=0),
    # layers.GRU(64, return_sequences=True, dropout=0.2),
    layers.Dense(32, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='mse')

es = callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True)

history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[es], verbose=0)

# 7) Evaluate (invert scaling to meaningful units)
y_pred_test_scaled = model.predict(test_ds).reshape(-1, 1)
y_pred_test = y_scaler.inverse_transform(y_pred_test_scaled).ravel()

# Align true test y and invert
y_true_test = y_scaler.inverse_transform(y_test).ravel()

mse = mean_squared_error(y_true_test, y_pred_test)
r2   = r2_score(y_true_test, y_pred_test)

print(f"Test RMSE: {rmse:.3f}")
print(f"Test MAE : {mae:.3f}")
print(f"Test R^2 : {r2:.3f}")
